In [1]:
"""
This Jupyter notebook implements the code from the
paper "Unveiling the Complex Morphologies of Sessile Droplets on Heterogeneous Surfaces"
for a flower-pattern substrate.
The notebook defines several global constants that describe the shape of the substrate.
These constants typically do not need to be modified.
A good entry point for the code is the function "radius".
This function calculates the radius matrix for a given droplet volume v.
The radius matrix is then processed by function " Height_volume",
which computes the droplet height H and the droplet surface area under specific m and n constraints.
By looping over a range of m and n values for a given droplet volume,
the surface area matrix is obtained through function "Main". The minimum value in the surface area matrix,
S_min, is identified, along with the corresponding parameters m and n.

Results:
The minimum surface area S_min and the corresponding parameters m and n
are identified, providing insights into the morphology of sessile droplets on flower-pattern substrat.
"""

'\nThis Jupyter notebook implements the code from the\npaper "Unveiling the Complex Morphologies of Sessile Droplets on Heterogeneous Surfaces"\nfor a flower-pattern substrate.\nThe notebook defines several global constants that describe the shape of the substrate.\nThese constants typically do not need to be modified.\nA good entry point for the code is the function "radius".\nThis function calculates the radius matrix for a given droplet volume v.\nThe radius matrix is then processed by function " Height_volume",\nwhich computes the droplet height H and the droplet surface area under specific m and n constraints.\nBy looping over a range of m and n values for a given droplet volume,\nthe surface area matrix is obtained through function "Main". The minimum value in the surface area matrix,\nS_min, is identified, along with the corresponding parameters m and n.\n\nResults:\nThe minimum surface area S_min and the corresponding parameters m and n\nare identified, providing insights into 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
from scipy.optimize import fsolve 
import sys
import time
from scipy import optimize
from numpy import savetxt

In [3]:
global Dimensions, PI, R_0, DEGREE, P, Q, A
Dimensions = 400
Dimensions_1 = 200
Dimensions_2= 200
PI = np.pi
R_0 = 0.56085639 # Initial parameter R0 in paper to ensure the substrate area equals to 1
DEGREE = 2
P=0.5 # The parameter "p" in Equation S22 
Q=4 # The parameter "q" in Equation S22 
A=0.05 #Amplitude Parameter "A" for perturbation term Rb in Equation S22

In [4]:
C_RANGES= np.reshape(np.linspace(1e-12, 0.999999, Dimensions), (-1,1))
#a1=np.linspace(1e-12,0.5-(0.999-0.5)/Dimensions_2, Dimensions_1) 
#a2=np.linspace(0.5,0.999, Dimensions_2) 
#C_RANGES= np.reshape(np.concatenate((a1, a2), axis=0), (-1,1))
THETA_RANGES = np.linspace(0, 2*PI, Dimensions)

In [5]:
def pertubation(theta):
    """
    Calculates the perturbation term Rb in Equation S22.

    Args:
        theta (array): The angle in the polar coordinate system..
        
    Returns:
        Rb (array): the perturbation term Rb in Equation S22.
    """
    return (np.cos(3*theta)+np.cos(8*theta)+np.sin(13*theta))

In [6]:
def dev_pertubation(theta):
    """
    Calculates the partial derivatives of perturbation term Rb with respect to angle theta

    Args:
        theta (array): The angle in the polar coordinate system..
        
    Returns:
        (array): the partial derivatives of perturbation term Rb with respect to angle theta

    """
    return 13*np.cos(13*theta)-8*np.sin(8*theta)-3*np.sin(3*theta)

In [7]:
def radius(theta, c, m):
    """
    This function generates a radius matrix.

    Args:
        theta (array-like): An array of angle ranges from 0 to 2π with a length of "dimension_theta".
        c (array-like): An array of the parameter "c" ranging from 0 to 1 with a length of "dimension_c".
        m (float): A scaling factor parameter "f(c)" as described in the referenced paper.

    Returns:
        numpy.ndarray: A radius matrix of size "dimension_theta" x "dimension_c" for the specified parameter "m".
    """
    global Dimensions, PI, R_0, DEGREE, P, Q, A
    pertu = pertubation(theta)
    scalling = 1/(np.power((1+c), m))
    r = scalling*((1-c)**P*R_0)+(np.power(1-c, Q)*A*pertu)###use R_0 = 0.561 to sure the surface = 1
    return r

In [8]:
def Dev_r_theta(theta, c, m):
    """
    Calculates the partial derivatives of the radius with respect to the angle
    (\(\partial R / \partial \theta\)), as described in Equation S23.

    Args:
        theta (array-like): Array of angle ranges from 0 to \(2\pi\), 
        with a length specified by the "dimension_theta".
        c_ranges (array-like): Array of parameter "c" ranges from 0 to 1,
        with a length specified by the "dimension_c".
        m (float): Scaling factor parameter "f(c)" as described in the paper.
    
    Returns:
        numpy.ndarray: Matrix of partial derivatives of the radius with respect to the angle,
        i.e. in Equation S23.
    """
    global Dimensions, PI, R_0, DEGREE, P, Q, A
    dev_per = dev_pertubation(theta)
    d_r = A*np.power((1-c), Q)*dev_per
    return d_r

In [9]:
def Dev_r_c(theta, c, m):
    """
    Calculates the partial derivatives of the radius with respect to the parameter (c)
    (\(\partial R / \partial c\)), as described in Equation S24.

    Args:
        theta (array-like): Array of angle ranges from 0 to \(2\pi\), 
        with a length specified by the "dimension_theta".
        c_ranges (array-like): Array of parameter "c" ranges from 0 to 1,
        with a length specified by the "dimension_c".
        m (float): Scaling factor parameter "f(c)" as described in the paper.
    
    Returns:
        numpy.ndarray: Matrix of partial derivatives of the radius with respect to the parameter (c),
        i.e. in Equation S24.
    """
    global Dimensions, PI, R_0, DEGREE, P, Q, A
    pertu = pertubation(theta)*A
    A = 1/(np.power((1+c), m))
    B = (1-c)**P*R_0
    dA = -m*np.power(1+c, -m-1)
    dB = (-P)*R_0*(1-c)**(P-1)
    dC = pertu*(Q)*((1-c)**((Q)-1))####require C != 1, when C == 1, there will be a signarity.
    d_r = A*dB+B*dA+dC
    return d_r

In [10]:
def Dev_h_c(c, n):
    """
    This function computes the partial derivative of height (h) with respect to c,
    denoted as (\partial h)/(\partial c), which corresponds to the third part in Eq.S9 but without info of "H".

    Args:
        c (array): Array of parameter values ranging from 0 to c_max
        with 'dimension' number of elements.
        n (float): Parameter used in the droplet height definition h = H * c^n in the paper,
        where H is the droplet height.

    Returns:
    (array):Array of partial derivatives of height with respect to parameter c,
    scaled by a constant factor of H.
    """
    global Dimensions, PI, R_0, DEGREE, P, Q, A
    d_h = n*np.power(c, n-1)####set c_max=1
    return d_h

In [11]:
def Height_volume(v, n, M, Radius, Theta_ranges, C_ranges):
    """
    Calculates the height and the surface area of a droplet given its volume 'v' and parameters 'n' and 'm'.

    Args:
        v (float): Droplet volume.
        m (float): Scaling factor parameter 'f(c)' as defined in the paper.
        n (float): Parameter defining the droplet height 'h = Hc^n' in the paper,
        where 'H' is the droplet height.
        Radius (ndarray): Matrix of radii corresponding to parameter 'm'.
        theta_ranges (ndarray): Array defining angle ranges from 0 to 2π with 'dimension_theta' length.
        C_ranges (ndarray): Array defining parameter 'c' ranges from 0 to 1 with 'dimension_c' length.

    Returns:
        H (float): Droplet height under parameters 'n' and 'm'.
        surface_size (float): Droplet surface area under parameters 'n' and 'm'.
    """
    dimen_c = len(C_ranges)
    
    d_theta = Theta_ranges[1]-Theta_ranges[0]#(Theta_ranges[-1]-Theta_ranges[0])/Dimensions
    C_RANGES_diff = np.diff(C_ranges, axis=0)
    C_RANGES_diff = np.reshape(np.append(C_RANGES_diff, C_RANGES_diff[-1,0] ), (-1,1))
    d_c = C_RANGES_diff
    d_base_area = 0.5*(Radius**2)*d_theta


    d_h = Dev_h_c(C_ranges, n)
    dv = (d_base_area*d_h)*d_c

    volume = np.sum(dv)

    H = v/volume
     
    d_r_theta = Dev_r_theta(Theta_ranges, C_ranges, M)
    d_r_c = Dev_r_c(Theta_ranges, C_ranges, M)
  
    d_h_c = H*d_h
    
    
    ds = np.sqrt(Radius**2+d_r_theta**2)*d_theta
    dl = np.sqrt(d_r_c**2+d_h_c**2)*d_c
    
    surface = ds*dl

    
    surface_size = np.sum(surface)
    
    return H, surface_size
    

In [12]:
def Main(v, n_ranges, m_ranges, Theta_ranges, C_ranges):
    """
    Calculates the minimum droplet surface area for a given volume.
    
    Args:
        v (float): Droplet volume.
        n_ranges (array-like): Array of possible values for parameter "n".
        m_ranges (array-like): Array of possible values for parameter "m".
        Theta_ranges (ndarray): Array defining angle ranges from 0 to 2π with 'dimension_theta' length.
        C_ranges (ndarray): Array defining parameter 'c' ranges from 0 to 1 with 'dimension_c' length.
       
    Returns:
        surface_min (float): Minimum surface area of the droplet.
        height_min (float): Corresponding height when the minimum surface area is achieved.
        surface_matrix (ndarray): Surface area matrix under dimensions specified by m_ranges*n_ranges.
        m_ranges[index[0]] (float): Parameter "m" value corresponding to the minimum surface area.
        n_ranges[index[1]] (float): Parameter "n" value corresponding to the minimum surface area.
    """
    n_length = len(n_ranges)
    m_length = len(m_ranges)
    surface_matrix = np.zeros((m_length, n_length))
    height_matrix = np.zeros((m_length, n_length))
    for i in range(m_length):
        Radius = radius(Theta_ranges, C_ranges, m_ranges[i])
        for j in range(n_length):
            height, surface = Height_volume(v, n_ranges[j], m_ranges[i], Radius, Theta_ranges, C_ranges)
            surface_matrix[i][j] = surface
            height_matrix[i][j] = height
    surface_min = surface_matrix.min()
    index = np.where(surface_matrix==np.min(surface_matrix))
    height_min = height_matrix[index][0]
    return surface_min, height_min, surface_matrix, m_ranges[index[0]], n_ranges[index[1]]

In [13]:
# def loop_volume(v):
#     m_l=[1, 0.5,0.1,0.05,0.01,0.005,0.001]
#     n_l=[0.1,0.05,0.01,0.005,0.001,0.0005,0.0001]
#     levels = len(m_l)
#     n_start = 0.5
#     n_stop = 3
#     m_start = -10
#     m_stop = 20
#     i = 0 
#     Start = True
#     while (i<levels):
#         dimen = 200
#         if (Start==True):
#             m_ranges = np.arange(m_start, m_stop, m_l[i])
#             n_ranges = np.arange(n_start,n_stop, n_l[i])
#             Start = False
#         else:
#             pass
# #         print(m_ranges)
# #         print(n_ranges)
#         s_g, h_g,energy_g= Main(v,n_ranges, m_ranges, THETA_RANGES, C_RANGES)
#         index = np.where(energy_g == np.min(energy_g))
#         m = index[0][0]
#         n = index[1][0]
# #         print('i', i)
# #         print('m', m)
# #         print('n', n)
#         m_lst_index = len(m_ranges)-1
#         n_lst_index = len(n_ranges)-1
#         if (m ==0)|(m==m_lst_index)|(n==0)|(n==n_lst_index):
#             m_ranges = np.arange(m_ranges[m]-m_l[i], m_ranges[m]+2*m_l[i], m_l[i])
#             n_ranges = np.arange(n_ranges[n]-n_l[i], n_ranges[n]+2*n_l[i], n_l[i])
#         else:
#             if (i+1) <levels:
#                 m_ranges = np.arange(m_ranges[m]-m_l[i], m_ranges[m]+m_l[i]+m_l[i+1], m_l[i+1])
#                 n_ranges = np.arange(n_ranges[n]-n_l[i], n_ranges[n]+n_l[i]+n_l[i+1], n_l[i+1])
#                 i = i+1
#             else:
#                 break
#     surface = s_g
#     height = h_g
#     Energy = energy_g
#     return surface, height, Energy,  m_ranges[m], n_ranges[n]

In [32]:
m_ranges = np.arange(-1, 1, 0.01)
n_ranges = np.arange(0.6,2, 0.01)
Volume = [0.1]#,0.5,1]#[0.06,0.10,0.14, 0.18,0.5, 1, 1.5]
Surface = []
Height = []
Energy = []
m_matrix = []
n_matrix = []
for v in Volume:
    s,h,e,m,n = Main(v, n_ranges, m_ranges, THETA_RANGES, C_RANGES)
    Surface.append(s)
    Height.append(h)
    Energy.append(e)
    m_matrix.append(m)
    n_matrix.append(n)

In [15]:
# m_ranges = np.linspace(-8, 10, 800)
# n_ranges = np.linspace(0.001, 4, 160)
# #########ACCORDING to the calculation from Square_with_gravity, m in the range of (0.975,1.952),n in the range of (1.189,1.561);
# Volume = [0.02,0.06,0.10,0.14, 0.18,0.5, 1, 1.5]
# Surface = []
# Height = []
# Energy = []
# m_matrix = []
# n_matrix = []
# for v in Volume:
#     s,h,e = Main(v, n_ranges, m_ranges, THETA_RANGES, C_RANGES)
#     Surface.append(s)
#     Height.append(h)
#     Energy.append(e)

In [16]:
#################THEO######################
Height

[0.0010399352326969965]

In [17]:
##############THEO#################
Surface#####

[1.9169818799698077]

In [18]:
m_matrix

[array([-1.])]

In [19]:
n_matrix

[array([0.6])]

In [20]:
surface_energy = Energy[0]

In [21]:
#savetxt('flower01.csv', surface_energy, delimiter=',')